In [1]:
from global_vars import *
from scenario import *
from pathlib import Path
import numpy as np

# Scene XML generation

In [2]:
scene_filepath = r"temp\scene.xml"
obj_filepath = Path(helios_dirpath, "data/sceneparts/Delft/9-276-556-LoD22-3D.obj")
tiff_filepath = Path(helios_dirpath, "data/sceneparts/Delft/M5_37EN1_5_m_filled.TIF")
tiff_mat_filepath = Path(helios_dirpath, "data/sceneparts/Delft/M5_37EN1_5_m_filled.TIF.mtl")

In [3]:
scene_part_obj = ScenePartOBJ(obj_filepath, up_axis="z")
scene_part_tiff = ScenePartTIFF(tiff_filepath, tiff_mat_filepath, "ground")

In [4]:
scene = Scene(xml_id="Delft", name="Delft", filepath=scene_filepath)
scene.add_scene_parts([scene_part_tiff, scene_part_obj])

In [5]:
scene.create_xml_string()
print(scene.xml_string)

<?xml version="1.0" encoding="UTF-8"?>
<document>
    <scene id="Delft" name="Delft">
        
        <part>
            <filter type="geotiffloader">
                <param type="string" key="filepath" value="C:\Users\Florian\Apps\helios-plusplus-win\data\sceneparts\Delft\M5_37EN1_5_m_filled.TIF" />
                <param type="string" key="matfile" value="C:\Users\Florian\Apps\helios-plusplus-win\data\sceneparts\Delft\M5_37EN1_5_m_filled.TIF.mtl" />
                <param type="string" key="matname" value="ground" />
            </filter>
            
        </part>

        <part>
            <filter type="objloader">
                <param type="string" key="filepath" value="C:\Users\Florian\Apps\helios-plusplus-win\data\sceneparts\Delft\9-276-556-LoD22-3D.obj" />
                <param type="string" key="up" value="z" />
            </filter>
            
            
        </part>
    </scene>
</document>


In [ ]:
scene.write_scene_file()

# Flight path generation

In [6]:
flight_path_filepath = r"temp\flight_path.xml"

def buffer_bbox(bbox, buffer_width = 50):
    bbox_buffer = np.array([-buffer_width, -buffer_width, buffer_width, buffer_width])
    bbox = bbox + bbox_buffer
    width = bbox_float[2] - bbox_float[0]
    height = bbox_float[3] - bbox_float[1]
    print(f"Width:  {width}")
    print(f"Height: {height}")
    return bbox

# bbox as [West, South, East, North]
bbox_float = [82585.1719, 445871.2500, 83613.3906, 446896.7813]
bbox_float = buffer_bbox(bbox_float, buffer_width=50)
bbox_int = np.round(bbox_float).astype(int)

strip_spacing = (bbox_int[3] - bbox_int[1]) / 5
altitude = 500
platform_speed = 60

Width:  1028.2186999999976
Height: 1025.5312999999733


In [7]:
flight_path = FlightPath(flight_path_filepath, bbox_int, strip_spacing, altitude, platform_speed)

In [8]:
flight_path.compute_waypoints()

In [9]:
flight_path.create_element_tree()

In [9]:
list(flight_path.tree.getroot().iter())

[<Element 'survey' at 0x0000021CA4D9F6F0>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9F240>,
 <Element 'leg' at 0x0000021CA93FA070>,
 <Element 'platformSettings' at 0x0000021CA4D9EC50>,
 <Element 'scannerSettings' at 0x0000021CA4D9EC00>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9E930>,
 <Element 'leg' at 0x0000021CA4D9ED40>,
 <Element 'platformSettings' at 0x0000021CA4D9EBB0>,
 <Element 'scannerSettings' at 0x0000021CA4D9EB10>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9E9D0>,
 <Element 'leg' at 0x0000021CA4D9EDE0>,
 <Element 'platformSettings' at 0x0000021CA4D9E7A0>,
 <Element 'scannerSettings' at 0x0000021CA4D9EA70>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9ECF0>,
 <Element 'leg' at 0x0000021CA4D9ECA0>,
 <Element 'platformSettings' at 0x0000021CA4D9E7F0>,
 <Element 'scannerSettings' at 0x0000021CA4D9EE80>,
 <Element <function Comment at 0x0000021CA65325F0> at 0x0000021CA4D9F1A0>,
 <

In [10]:
flight_path.create_xml_string()
print(flight_path.xml_string)

<survey>
    <!--leg 000-->
    <leg>
        <platformSettings x="83663.0" y="445821.0" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" />
    </leg>
    <!--leg 001-->
    <leg>
        <platformSettings x="82535.0" y="445821.0" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" active="false" />
    </leg>
    <!--leg 002-->
    <leg>
        <platformSettings x="82535.0" y="446046.2" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" />
    </leg>
    <!--leg 003-->
    <leg>
        <platformSettings x="83663.0" y="446046.2" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" active="false" />
    </leg>
    <!--leg 004-->
    <leg>
        <platformSettings x="83663.0" y="446271.4" z="500" movePerSec_m="60" />
        <scannerSett

In [11]:
flight_path.write_flight_path_file()

# Survey generation

In [10]:
survey_filepath = r"temp\survey.xml"
platform_xml_path = str(Path(helios_dirpath, "data/platforms.xml#sr22"))
scanner_xml_path = str(Path(helios_dirpath, "data/scanners_als.xml#riegl_vq_780i"))

survey = Survey(
    name="Delft",
    filepath=survey_filepath,
    scene=scene,
    platform=platform_xml_path,
    scanner=scanner_xml_path,
    flight_path=flight_path #_filepath
)

In [11]:
survey.populate_template()

In [12]:
survey.write_survey_file()

# Generating element tree from scratch

In [5]:
import xml.etree.ElementTree as eT

In [6]:
root = eT.Element("survey")
tree = eT.ElementTree(root)

In [7]:
flight_path.waypoints

array([[ 83663. , 445821. ],
       [ 82535. , 445821. ],
       [ 82535. , 446046.2],
       [ 83663. , 446046.2],
       [ 83663. , 446271.4],
       [ 82535. , 446271.4],
       [ 82535. , 446496.6],
       [ 83663. , 446496.6],
       [ 83663. , 446721.8],
       [ 82535. , 446721.8],
       [ 82535. , 446947. ],
       [ 83663. , 446947. ]])

In [8]:
short_leg_active = "true" if flight_path.always_active else "false"

for i, leg in enumerate(flight_path.waypoints):
    root.append(eT.Comment(f"leg {i:03}"))
    
    leg_element = eT.Element("leg")
    root.append(leg_element)
    
    platform_settings_element = eT.Element(
        "platformSettings",
        attrib={
            "x": str(leg[0]),
            "y": str(leg[1]),
            "z": str(flight_path.altitude),
            "movePerSec_m": str(flight_path.velocity)
        }
    )
    leg_element.append(platform_settings_element)

    scanner_settings_element = eT.Element(
        "scannerSettings",
        attrib={
            "template": str(flight_path.scanner_settings_id),
            "trajectoryTimeInterval_s": str(flight_path.trajectory_time_interval)
        }
    )
    if i % 2 != 0:
        scanner_settings_element.attrib["active"] = str(short_leg_active)
    leg_element.append(scanner_settings_element)    

In [9]:
eT.indent(tree, "    ")

In [20]:
print(eT.tostring(tree.getroot(), encoding="unicode", xml_declaration=False))

<survey>
    <!--leg 000-->
    <leg>
        <platformSettings x="83663.0" y="445821.0" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" />
    </leg>
    <!--leg 001-->
    <leg>
        <platformSettings x="82535.0" y="445821.0" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" active="false" />
    </leg>
    <!--leg 002-->
    <leg>
        <platformSettings x="82535.0" y="446046.2" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" />
    </leg>
    <!--leg 003-->
    <leg>
        <platformSettings x="83663.0" y="446046.2" z="500" movePerSec_m="60" />
        <scannerSettings template="scanner_settings" trajectoryTimeInterval_s="0.05" active="false" />
    </leg>
    <!--leg 004-->
    <leg>
        <platformSettings x="83663.0" y="446271.4" z="500" movePerSec_m="60" />
        <scannerSett

In [29]:
tree.write(r"temp\flight_path_etree.xml", encoding="UTF-8", xml_declaration=False)